# this program reads angles from images and stores ENUMs in database

In [1]:
import os
from tqdm import tqdm
from sql_face.sqldb import SQLDataBase

home = os.path.expanduser("~")

db_name= 'test_1' #'colab_main_dataset'
input_dir = os.path.join(home, 'video_resources', 'image_datasets')
output_dir = os.path.join(home, 'video_resources', 'sql_database')
database_names=['lfw'] # 'lfw', 'scface',  'enfsi', 'enfsi2015', , 'utkface','chokepoint','forenface'
detector_names=['mtcnn'] # 'dlib', 'mtcnn', 'mtcnn_serfiq'
embedding_model_names=["ArcFace"] # , "Dlib", "ArcFace",
attributes_to_update=['yaw','pitch','roll']  # 'gender', 'age', 'emotion', 'race'
quality_model_names = ['ser_fiq', 'tface'] # 'confusion_score', 'ser_fiq',
save_in_drive = False # To work in google colab



2023-03-18 15:13:46.221369: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [2]:
database = SQLDataBase(db_name=db_name,
                        input_dir = input_dir,
                        output_dir_name = output_dir,
                        database_names = database_names,
                        detector_names = detector_names,
                        embedding_model_names = embedding_model_names,
                        quality_model_names = quality_model_names)

# thresholds calculated previously (excel)

In [3]:
th = {
'pitch': [-16.47307851, -7.950750253, 0.710368788, 6.941161856, 99999],
'yaw': [10.84743824,
20.52868443,
99999],
'roll': [ 
6.373882696,
13.32788435,
99999]}

In [6]:
from face_parser import FaceParser
from sql_face.tables import Image, EnfsiImage, VideoFrame, EnfsiVideoFrame, Yaw, Pitch, Roll

session = database.session
face_parser = FaceParser()

for db in database.databases:
    
    images = (session.query(Image).filter(Image.roll == None).all())
#images = (session.query(Image).filter(Image.yaw == None).all())

    n1 = 100
    n= n1
    for image in tqdm(images,desc = f'Procesando {db.source}'):
        img = image.get_image(input_dir)
        faces, landmarks, angles = face_parser.detect_faces_lms_ang(img)
        if angles:
            pitch = angles[0][0]
            yaw = abs(angles[0][1])
            roll = abs(angles[0][2])

            if not image.roll:
                image.roll = next(X_enum for X_enum, umbral in zip(Roll, th['roll']) if roll < umbral)  
            if not image.yaw:          
                image.yaw = next(X_enum for X_enum, umbral in zip(Yaw, th['yaw']) if yaw < umbral)
            if not image.pitch:
                image.pitch = next(X_enum for X_enum, umbral in zip(Pitch, th['pitch']) if pitch < umbral)
            if n:
                n -=1
            else:
                session.commit()
                n = n1
                
                
        session.commit()
    session.commit()

            

Procesando LFW:   0%|          | 14/30779 [00:09<2:27:06,  3.49it/s]